In [126]:
import pyautogui as pg
from PIL import ImageGrab, ImageOps
import time
import numpy as np
import cv2

board_val = 0
retry = 0
LIFE = 0
R_VAL = 1455 #normalized value sum of retry button pixels 1455
reward = 1
states = [0]

#Q-matrix
old_q = np.zeros((20,3)) 
q = np.zeros((20,3))
for i in range(q.shape[0]):
    q[i,] = [0 ,1 ,1.5]
    np.random.shuffle(q[i,])
    old_q[i,] = q[i,]


class Cordinates():
    #You may need to  tweak these according your screen resolution
    replay = (380,319,421,349) #replay button coordinates
    replayBtn = (397,336) #approx center coordinates for replay button
    dino = (85,352) #Dino coordinates Standing(86,290) sitting(105,313)
    box = (87,363,90,377) #A rect in-front of our Agent 
def restart():
    pg.click(Cordinates.replayBtn)

def give_reward():
    global reward
    global R_VAL
    c = Cordinates()
    replay = ImageGrab.grab(c.replay) #get replay button position
    replay = np.array(replay)
    if int(replay.sum()/255) == R_VAL: #if replay button is seen then a life has beeen consumed
        reward = 0 #reward 0 when dead
    else:
        reward = 1
def do_none():
    pg.keyDown("left")
    time.sleep(0.1)
    print("Do nothing")
    pg.keyUp("left")
    give_reward()
def jump():
    pg.keyDown("space")
    time.sleep(0.2)
    print("Jump")
    pg.keyUp("space")
    give_reward()


def duck(): #crouch
    
    pg.keyDown("down")
    time.sleep(0.1)
    print("duck")
    pg.keyUp("down")
    give_reward()

def imgGrab():
    global board_val
    global retry
    global reward
    global R_VAL
    c = Cordinates()
    replay = ImageGrab.grab(c.replay) #get replay button position
    replay = np.array(replay)
    if int(replay.sum()/255) == R_VAL: #if replay button is seen then a life has beeen consumed
        print("Life consumed")
        retry += 1

    #print(reward)
    img = ImageGrab.grab(c.box)
    gray = ImageOps.grayscale(img)
    a = np.array(gray.getcolors())
    return a.sum()

In [128]:
gamma = 0.8 #Discount factor
score = 0 # Initial score value
states = np.array(states)
LIFE = 20 #no.of lifes for our Agent(Dino)

def choose_action(state):
    q_row_index = np.where(state == states)[0][0] #Get the row with the state
    action = np.argmax(q[q_row_index,])
    return action

def update_q(current_state,action,reward):
    
    #Q-Learning algorithm: 
    #Q[current_state,action] = rewards[current_state,action] + gamma * max[Q[current_state,action]]    
    
    
    
    global gamma
    global alpha
    state_index = np.where(current_state == states)[0][0]
    max_val = np.max(q[state_index,]) #This is the action we took
    q[state_index,action] = reward + gamma * max_val # Algorithm

def run():
    global states
    global score
    global q
    global reward
    while True:
        s = imgGrab()
        #add new states to the states table
        if(len(np.where(states == s)[0]) < 1):
            states = np.append(states,s)

        #print(reward)

#         print(s)
        if retry == 1:
            #print("Game Over")
            break
        if s != 2895 and s!=2640:
            try:
                act = choose_action(s)
                print("action for {} is {}".format(s,act))
            except Exception as e:
                pass
            score += 1
            #print(s)

            #Perform action performed based on Q-table
            #actions = {0:do nothing ,1:jump ,2:duck}
            if act == 0:
                do_none()
                print(reward)
                update_q(s,act,reward)
            elif act == 1:
                jump()
                print(reward)
                update_q(s,act,reward)
            elif act == 2:
                duck()
#                 print(reward)
                print(reward)
                update_q(s,act,reward)
            time.sleep(0.1)
            
        if score == 500:
            print("Score Reached")
            break

for i in range(LIFE):
    if score == 500:
        break
    else:
        print("Score of run:",score)
        score = 0
        print("Life:",i)
        retry = 0
        restart()
        run()


Score of run: 0
Life: 0
action for 297 is 1
Jump
1
action for 297 is 1
Jump
1
action for 297 is 1
Jump
1
action for 297 is 1
Jump
1
action for 297 is 1
Jump
1
action for 297 is 1
Jump
1
action for 297 is 1
Jump
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1
action for 380 is 0
Do nothing
1


KeyboardInterrupt: 

In [129]:
def stats(): #prints stats like Q-matrix and states
    print("len & num states:",len(states),states)

    print("Q-matrix:\n",q[:len(states)])

stats()
print("\n\n initial Q-matrix\n",old_q[:len(states)] )

# In-case u want to save ur Q-matrix
np.save("Dino_Q_matrix.npy",q)

len & num states: 4 [  0 297 629 380]
Q-matrix:
 [[1.5        1.         0.        ]
 [1.         4.99962782 0.        ]
 [0.         3.208      1.        ]
 [4.99653377 1.         0.        ]]


 initial Q-matrix
 [[1.5 1.  0. ]
 [1.  1.5 0. ]
 [0.  1.5 1. ]
 [1.5 1.  0. ]]


In [139]:
from PIL import ImageGrab, ImageOps
import pyautogui
import time
import numpy as np  
   
class coordinates():
    # coordinates of replay button to start the game
    replaybutton =(397, 336)
    # this coordinates represent the top-right coordinates
    # that will be used to define the front box
    dinasaur = (90, 360) 
def restartGame():
 
    # using pyautogui library, we are clicking on the 
    # replay button without any user interaction 
    pyautogui.click(coordinates.replaybutton)
    # we will keep our Bot always down that 
    # will prevent him to get hit by bird 
    pyautogui.keyDown('down')

In [143]:
def press_space():
  
    # releasing the Down Key 
    pyautogui.keyUp('down') 
 
    # pressing Space to overcome Bush
    pyautogui.keyDown('space')
 
    # so that Space Key will be recognized easily
    time.sleep(0.1) 
 
    # printing the "Jump" statement on the
    # terminal to see the current output 
    print("jump")
    time.sleep(0.10)
 
    # releasing the Space Key 
    pyautogui.keyUp('space')
 
    # again pressing the Down Key to keep my Bot always down 
    pyautogui.keyDown('down')
 


In [152]:
def imageGrab(): 
    # defining the coordinates of box in front of dinosaur 
    box = (coordinates.dinasaur[0]+30, coordinates.dinasaur[1],
           coordinates.dinasaur[0]+50, coordinates.dinasaur[1]+2)
 
    # grabbing all the pixels values in form of RGB tupples   
    image = ImageGrab.grab(box)
 
    # converting RGB to Grayscale to
    # make processing easy and result faster 
    grayImage = ImageOps.grayscale(image)
 
    # using numpy to get sum of all grayscale pixels 
    a = np.array(grayImage.getcolors())
 
    # returning the sum
    print(a.sum()) 
    return a.sum()


In [157]:
# function to restart the game
restartGame()
while True: 
     # 435 is the sum of white pixels values of box. 
     # You may get different value is you are taking bigger 
     # or smaller box than the box taken in this article. 
     # if value returned by "imageGrab" function is not equal to 435, 
     # it means either bird or bush is coming towards dinosaur 
    if(imageGrab() != 627):   
        press_space()    
        # time to recogn ize the operation performed by above function 
        time.sleep(0.1) 

295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump
295
jump


KeyboardInterrupt: 